# Import Libraries

In [1]:
import pandas as pd 
import numpy as np
import re
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import StandardScaler
pd.set_option('display.max_colwidth', None) 

# Loading Dataframe

In [2]:
df=pd.read_csv('../data/Westelm_data.csv')

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4257 entries, 0 to 4256
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Title          4257 non-null   object
 1   Link           4257 non-null   object
 2   Sale Price     4257 non-null   object
 3   Regular Price  4257 non-null   object
 4   Image Path     4257 non-null   object
 5   Swatch Colors  3165 non-null   object
dtypes: object(6)
memory usage: 199.7+ KB


In [4]:
df.head()

,Title,Link,Sale Price,Regular Price,Image Path,Swatch Colors
0,"Haven 2-Piece Bumper Chaise Sectional (106""–113"")",https://www.westelm.com/products/haven-2-piece-terminal-chaise-sectional-h3422/?pkey=csectionals,"2,633.30","4,698","images/Haven_2-Piece_Bumper_Chaise_Sectional_(106""–113"").jpg","Performance Yarn Dyed Linen Weave, Alabaster, Performance Yarn Dyed Linen Weave, Pewter, Performance Yarn Dyed Linen Weave, Graphite, Performance Yarn Dyed Linen Weave, Frost Gray"
1,"Laurent 2-Piece Wedge Chaise Sectional (111.5"")",https://www.westelm.com/products/laurent-2-piece-wedge-chaise-sectional-h12004/?pkey=csectionals,"2,399","2,999","images/Laurent_2-Piece_Wedge_Chaise_Sectional_(111.5"").jpg","Performance Yarn Dyed Linen Weave, Alabaster, Performance Yarn Dyed Linen Weave, Frost Gray, Performance Yarn Dyed Linen Weave, Sand, Yarn Dyed Linen Weave, Graphite"
2,"Marin 2-Piece Bumper Chaise Sectional (114"")",https://www.westelm.com/products/marin-terminal-chaise-sectional-h6956/?pkey=csectionals,"2,309.99","4,098","images/Marin_2-Piece_Bumper_Chaise_Sectional_(114"").jpg","Performance Yarn Dyed Linen Weave, Alabaster, Performance Yarn Dyed Linen Weave, Graphite, Performance Yarn Dyed Linen Weave, Frost Gray, Performance Yarn Dyed Linen Weave, Sand"
3,"Harris 2-Piece Bumper Chaise Sectional (100""–116"")",https://www.westelm.com/products/harris-2-piece-terminal-sectional-h4618/?pkey=csectionals,"2,198","3,598","images/Harris_2-Piece_Bumper_Chaise_Sectional_(100""–116"").jpg","Performance Yarn Dyed Linen Weave, Alabaster, Performance Yarn Dyed Linen Weave, Graphite, Performance Yarn Dyed Linen Weave, Frost Gray, Performance Yarn Dyed Linen Weave, Midnight"
4,"Harmony 2-Piece Chaise Sectional (112""–128"")",https://www.westelm.com/products/harmony-2-piece-chaise-sectional-h2478/?pkey=csectionals,"3,228.30","5,098","images/Harmony_2-Piece_Chaise_Sectional_(112""–128"").jpg","Performance Yarn Dyed Linen Weave, Alabaster, Performance Yarn Dyed Linen Weave, Graphite, Performance Yarn Dyed Linen Weave, Frost Gray, Yarn Dyed Linen Weave, Alabaster"


In [5]:
df.isna().sum()

Title               0
Link                0
Sale Price          0
Regular Price       0
Image Path          0
Swatch Colors    1092
dtype: int64

In [6]:
df = df[['Title','Sale Price','Swatch Colors','Link']]

In [7]:
df.isna().sum()

Title               0
Sale Price          0
Swatch Colors    1092
Link                0
dtype: int64

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4257 entries, 0 to 4256
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Title          4257 non-null   object
 1   Sale Price     4257 non-null   object
 2   Swatch Colors  3165 non-null   object
 3   Link           4257 non-null   object
dtypes: object(4)
memory usage: 133.2+ KB


In [9]:
df.rename(columns={'Sale Price': 'Price',
                   'Swatch Colors': 'Colors'},
          inplace=True, errors='raise')

# Displaying Dataset

In [10]:
df.head()

,Title,Price,Colors,Link
0,"Haven 2-Piece Bumper Chaise Sectional (106""–113"")","2,633.30","Performance Yarn Dyed Linen Weave, Alabaster, Performance Yarn Dyed Linen Weave, Pewter, Performance Yarn Dyed Linen Weave, Graphite, Performance Yarn Dyed Linen Weave, Frost Gray",https://www.westelm.com/products/haven-2-piece-terminal-chaise-sectional-h3422/?pkey=csectionals
1,"Laurent 2-Piece Wedge Chaise Sectional (111.5"")","2,399","Performance Yarn Dyed Linen Weave, Alabaster, Performance Yarn Dyed Linen Weave, Frost Gray, Performance Yarn Dyed Linen Weave, Sand, Yarn Dyed Linen Weave, Graphite",https://www.westelm.com/products/laurent-2-piece-wedge-chaise-sectional-h12004/?pkey=csectionals
2,"Marin 2-Piece Bumper Chaise Sectional (114"")","2,309.99","Performance Yarn Dyed Linen Weave, Alabaster, Performance Yarn Dyed Linen Weave, Graphite, Performance Yarn Dyed Linen Weave, Frost Gray, Performance Yarn Dyed Linen Weave, Sand",https://www.westelm.com/products/marin-terminal-chaise-sectional-h6956/?pkey=csectionals
3,"Harris 2-Piece Bumper Chaise Sectional (100""–116"")","2,198","Performance Yarn Dyed Linen Weave, Alabaster, Performance Yarn Dyed Linen Weave, Graphite, Performance Yarn Dyed Linen Weave, Frost Gray, Performance Yarn Dyed Linen Weave, Midnight",https://www.westelm.com/products/harris-2-piece-terminal-sectional-h4618/?pkey=csectionals
4,"Harmony 2-Piece Chaise Sectional (112""–128"")","3,228.30","Performance Yarn Dyed Linen Weave, Alabaster, Performance Yarn Dyed Linen Weave, Graphite, Performance Yarn Dyed Linen Weave, Frost Gray, Yarn Dyed Linen Weave, Alabaster",https://www.westelm.com/products/harmony-2-piece-chaise-sectional-h2478/?pkey=csectionals


# Droping Duplicate

In [11]:
df.drop_duplicates(subset = "Title",inplace=True)

In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2212 entries, 0 to 4123
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Title   2212 non-null   object
 1   Price   2212 non-null   object
 2   Colors  1599 non-null   object
 3   Link    2212 non-null   object
dtypes: object(4)
memory usage: 86.4+ KB


# Using Regex to Clean `Title` Column

In [13]:
df['Title'] = df['Title'].str.replace(r'\s*\([^)]*\)', '',regex=True)

In [14]:
df.head()

,Title,Price,Colors,Link
0,Haven 2-Piece Bumper Chaise Sectional,"2,633.30","Performance Yarn Dyed Linen Weave, Alabaster, Performance Yarn Dyed Linen Weave, Pewter, Performance Yarn Dyed Linen Weave, Graphite, Performance Yarn Dyed Linen Weave, Frost Gray",https://www.westelm.com/products/haven-2-piece-terminal-chaise-sectional-h3422/?pkey=csectionals
1,Laurent 2-Piece Wedge Chaise Sectional,"2,399","Performance Yarn Dyed Linen Weave, Alabaster, Performance Yarn Dyed Linen Weave, Frost Gray, Performance Yarn Dyed Linen Weave, Sand, Yarn Dyed Linen Weave, Graphite",https://www.westelm.com/products/laurent-2-piece-wedge-chaise-sectional-h12004/?pkey=csectionals
2,Marin 2-Piece Bumper Chaise Sectional,"2,309.99","Performance Yarn Dyed Linen Weave, Alabaster, Performance Yarn Dyed Linen Weave, Graphite, Performance Yarn Dyed Linen Weave, Frost Gray, Performance Yarn Dyed Linen Weave, Sand",https://www.westelm.com/products/marin-terminal-chaise-sectional-h6956/?pkey=csectionals
3,Harris 2-Piece Bumper Chaise Sectional,"2,198","Performance Yarn Dyed Linen Weave, Alabaster, Performance Yarn Dyed Linen Weave, Graphite, Performance Yarn Dyed Linen Weave, Frost Gray, Performance Yarn Dyed Linen Weave, Midnight",https://www.westelm.com/products/harris-2-piece-terminal-sectional-h4618/?pkey=csectionals
4,Harmony 2-Piece Chaise Sectional,"3,228.30","Performance Yarn Dyed Linen Weave, Alabaster, Performance Yarn Dyed Linen Weave, Graphite, Performance Yarn Dyed Linen Weave, Frost Gray, Yarn Dyed Linen Weave, Alabaster",https://www.westelm.com/products/harmony-2-piece-chaise-sectional-h2478/?pkey=csectionals


# Replacing Commas With Nothing in Price Column

In [15]:
df['Price'] = df['Price'].str.replace(',', '',regex=True)

In [16]:
df.head()

,Title,Price,Colors,Link
0,Haven 2-Piece Bumper Chaise Sectional,2633.30,"Performance Yarn Dyed Linen Weave, Alabaster, Performance Yarn Dyed Linen Weave, Pewter, Performance Yarn Dyed Linen Weave, Graphite, Performance Yarn Dyed Linen Weave, Frost Gray",https://www.westelm.com/products/haven-2-piece-terminal-chaise-sectional-h3422/?pkey=csectionals
1,Laurent 2-Piece Wedge Chaise Sectional,2399,"Performance Yarn Dyed Linen Weave, Alabaster, Performance Yarn Dyed Linen Weave, Frost Gray, Performance Yarn Dyed Linen Weave, Sand, Yarn Dyed Linen Weave, Graphite",https://www.westelm.com/products/laurent-2-piece-wedge-chaise-sectional-h12004/?pkey=csectionals
2,Marin 2-Piece Bumper Chaise Sectional,2309.99,"Performance Yarn Dyed Linen Weave, Alabaster, Performance Yarn Dyed Linen Weave, Graphite, Performance Yarn Dyed Linen Weave, Frost Gray, Performance Yarn Dyed Linen Weave, Sand",https://www.westelm.com/products/marin-terminal-chaise-sectional-h6956/?pkey=csectionals
3,Harris 2-Piece Bumper Chaise Sectional,2198,"Performance Yarn Dyed Linen Weave, Alabaster, Performance Yarn Dyed Linen Weave, Graphite, Performance Yarn Dyed Linen Weave, Frost Gray, Performance Yarn Dyed Linen Weave, Midnight",https://www.westelm.com/products/harris-2-piece-terminal-sectional-h4618/?pkey=csectionals
4,Harmony 2-Piece Chaise Sectional,3228.30,"Performance Yarn Dyed Linen Weave, Alabaster, Performance Yarn Dyed Linen Weave, Graphite, Performance Yarn Dyed Linen Weave, Frost Gray, Yarn Dyed Linen Weave, Alabaster",https://www.westelm.com/products/harmony-2-piece-chaise-sectional-h2478/?pkey=csectionals


# Replacing Commas with Nothing in Colors Column 

In [17]:
df['Colors']=df['Colors'].str.replace(',','')

In [18]:
df.head()

,Title,Price,Colors,Link
0,Haven 2-Piece Bumper Chaise Sectional,2633.30,Performance Yarn Dyed Linen Weave Alabaster Performance Yarn Dyed Linen Weave Pewter Performance Yarn Dyed Linen Weave Graphite Performance Yarn Dyed Linen Weave Frost Gray,https://www.westelm.com/products/haven-2-piece-terminal-chaise-sectional-h3422/?pkey=csectionals
1,Laurent 2-Piece Wedge Chaise Sectional,2399,Performance Yarn Dyed Linen Weave Alabaster Performance Yarn Dyed Linen Weave Frost Gray Performance Yarn Dyed Linen Weave Sand Yarn Dyed Linen Weave Graphite,https://www.westelm.com/products/laurent-2-piece-wedge-chaise-sectional-h12004/?pkey=csectionals
2,Marin 2-Piece Bumper Chaise Sectional,2309.99,Performance Yarn Dyed Linen Weave Alabaster Performance Yarn Dyed Linen Weave Graphite Performance Yarn Dyed Linen Weave Frost Gray Performance Yarn Dyed Linen Weave Sand,https://www.westelm.com/products/marin-terminal-chaise-sectional-h6956/?pkey=csectionals
3,Harris 2-Piece Bumper Chaise Sectional,2198,Performance Yarn Dyed Linen Weave Alabaster Performance Yarn Dyed Linen Weave Graphite Performance Yarn Dyed Linen Weave Frost Gray Performance Yarn Dyed Linen Weave Midnight,https://www.westelm.com/products/harris-2-piece-terminal-sectional-h4618/?pkey=csectionals
4,Harmony 2-Piece Chaise Sectional,3228.30,Performance Yarn Dyed Linen Weave Alabaster Performance Yarn Dyed Linen Weave Graphite Performance Yarn Dyed Linen Weave Frost Gray Yarn Dyed Linen Weave Alabaster,https://www.westelm.com/products/harmony-2-piece-chaise-sectional-h2478/?pkey=csectionals


In [19]:
df['Colors'] = pd.factorize(df['Colors'])[0]

In [20]:
df.head()

,Title,Price,Colors,Link
0,Haven 2-Piece Bumper Chaise Sectional,2633.30,0,https://www.westelm.com/products/haven-2-piece-terminal-chaise-sectional-h3422/?pkey=csectionals
1,Laurent 2-Piece Wedge Chaise Sectional,2399,1,https://www.westelm.com/products/laurent-2-piece-wedge-chaise-sectional-h12004/?pkey=csectionals
2,Marin 2-Piece Bumper Chaise Sectional,2309.99,2,https://www.westelm.com/products/marin-terminal-chaise-sectional-h6956/?pkey=csectionals
3,Harris 2-Piece Bumper Chaise Sectional,2198,3,https://www.westelm.com/products/harris-2-piece-terminal-sectional-h4618/?pkey=csectionals
4,Harmony 2-Piece Chaise Sectional,3228.30,4,https://www.westelm.com/products/harmony-2-piece-chaise-sectional-h2478/?pkey=csectionals


# Converting Price From String To Float

In [21]:
df['Price'] = df['Price'].str.replace(r'\S*[a-zA-Z]\S*', '0',regex=True)

In [25]:
df['Price'] = df['Price'].str.replace(' ', '')

In [30]:
df['Price']=df['Price'].astype('float')

In [31]:
scaler = StandardScaler()

In [32]:
df['Price'] = scaler.fit_transform(df[['Price']])

In [33]:
# Creating feature vectors
features = df[['Colors', 'Price']].values

# Compute cosine similarity matrix
similarity_matrix = cosine_similarity(features, features)

In [35]:
def recommend_items(item_name, similarity_matrix, df, top_n=10):
    if item_name not in df['Title'].values:
        return f"Item '{item_name}' not found in the dataset."

    item_index = df.index[df['Title'] == item_name].tolist()[0]

    similarity_scores = list(enumerate(similarity_matrix[item_index]))

    similarity_scores = sorted(similarity_scores, key=lambda x: x[1], reverse=True)

    top_similar_items = similarity_scores[1:top_n+1]

    recommended_items = [df.iloc[item[0]]['Title'] for item in top_similar_items]

    return recommended_items

# Example usage
item_name = 'Haven 2-Piece Bumper Chaise Sectional'
recommendations = recommend_items(item_name, similarity_matrix, df)
print(f"Because you viewed {item_name}, you may also like: {', '.join(recommendations)}")


Because you viewed Haven 2-Piece Bumper Chaise Sectional, you may also like: Harmony Modular 3-Piece Ottoman Sectional, Harris 2-Piece Sleeper Sectional w/ Bumper Chaise, Harmony Modular 2-Piece Sleeper Sectional w/ Storage Chaise, Harper 2-Piece Chaise Sectional, Harmony Modular 2-Piece Chaise Sectional, Parker 2-Piece Chaise Sectional, Nassau 2-Piece Chaise Sectional, Harris Skirted Slipcover 2-Piece Chaise Sectional, Harris 2-Piece Reversible Sectional, Harris 2-Piece Pop-Up Sleeper Sectional w/ Storage Chaise


In [36]:
recommendations

['Harmony Modular 3-Piece Ottoman Sectional',
 'Harris 2-Piece Sleeper Sectional w/ Bumper Chaise',
 'Harmony Modular 2-Piece Sleeper Sectional w/ Storage Chaise',
 'Harper 2-Piece Chaise Sectional',
 'Harmony Modular 2-Piece Chaise Sectional',
 'Parker 2-Piece Chaise Sectional',
 'Nassau 2-Piece Chaise Sectional',
 'Harris Skirted Slipcover 2-Piece Chaise Sectional',
 'Harris 2-Piece Reversible Sectional',
 'Harris 2-Piece Pop-Up Sleeper Sectional w/ Storage Chaise']

In [37]:
df[df['Title']=="Big Sur Leather Sectional Ottoman"]

,Title,Price,Colors,Link


In [38]:
df[df['Title']=="Big Sur Leather Sectional Ottoman"]['Link']

Series([], Name: Link, dtype: object)

In [39]:
[df[df['Title']==rec]['Link'] for rec in recommendations]

[5    https://www.westelm.com/products/harmony-modular-3-piece-chaise-sectional-h6269/?pkey=csectionals
 Name: Link, dtype: object,
 7    https://www.westelm.com/products/harris-sleeper-sectional-w-terminal-chaise-h4588/?pkey=csectionals
 Name: Link, dtype: object,
 8    https://www.westelm.com/products/harmony-modular-2-piece-sleeper-sectional-w-storage-h6268/?pkey=csectionals
 Name: Link, dtype: object,
 10    https://www.westelm.com/products/harper-2-piece-chaise-sectional-h6919/?pkey=csectionals
 Name: Link, dtype: object,
 17    https://www.westelm.com/products/harmony-modular-2-piece-chaise-sectional-h6267/?pkey=csectionals
 Name: Link, dtype: object,
 18    https://www.westelm.com/products/parker-2-piece-chaise-sectional-h9538/?pkey=csectionals
 Name: Link, dtype: object,
 24    https://www.westelm.com/products/nassau-2-piece-chaise-sectional-h12314/?pkey=csectionals
 Name: Link, dtype: object,
 25    https://www.westelm.com/products/harris-skirted-slipcover-2-piece-chaise-secti

In [ ]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import StandardScaler

# Example dataset (replace with your actual dataset loading)
data = {
    'furniture_name': ['Sofa', 'Table', 'Chair', 'Cushion', 'Bed', 'Desk'],
    'color': ['Red', 'Brown', 'White', 'Blue', 'Black', 'Brown'],
    'price': [500, 300, 150, 50, 800, 400]
}

df = pd.DataFrame(data)

# Encoding categorical variables (color)
df['color_encoded'] = pd.factorize(df['color'])[0]

# Scaling numerical features (price)
scaler = StandardScaler()
df['price_scaled'] = scaler.fit_transform(df[['price']])

# Creating feature vectors
features = df[['color_encoded', 'price_scaled']].values

# Compute cosine similarity matrix
similarity_matrix = cosine_similarity(features, features)

# Function to recommend items
def recommend_items(item_name, similarity_matrix, df, top_n=3):
    item_index = df.index[df['furniture_name'] == item_name][0]
    similarity_scores = list(enumerate(similarity_matrix[item_index]))
    similarity_scores = sorted(similarity_scores, key=lambda x: x[1], reverse=True)
    top_similar_items = similarity_scores[1:top_n+1]  # Exclude itself, get top N similar items
    recommended_items = [df.iloc[item[0]]['furniture_name'] for item in top_similar_items]
    return recommended_items

# Example usage
item_name = 'Sofa'
recommendations = recommend_items(item_name, similarity_matrix, df)
print(f"Because you viewed {item_name}, you may also like: {', '.join(recommendations)}")
